In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk import tokenize
import lda
import pyLDAvis
import pyLDAvis.gensim
import warnings
warnings.filterwarnings('ignore')
#import logging
import gensim
from gensim.models.ldamodel import LdaModel

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
lemma = WordNetLemmatizer()
STOPWORDS=set(stopwords.words('english')+list(punctuation))
STOPWORDS.remove("not")

In [3]:
df_Tpoics = pd.read_csv("data/proData/Topic.csv",  
                 dtype={'TweetsproDataID': str,'TweetsText':str,'User_Location':str,'Date':str})

In [4]:
df_Tpoics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6836 entries, 0 to 6835
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   TweetsID         6836 non-null   int64 
 1   Date             6836 non-null   object
 2   text             6836 non-null   object
 3   Sentiment_score  6836 non-null   object
dtypes: int64(1), object(3)
memory usage: 213.8+ KB


In [5]:
df_Tpoics.head(2)

,TweetsID,Date,text,Sentiment_score
0,1216910000000000000,24/10/2020 03:29,everything get students back school safely qui...,Positive
1,191554292,23/10/2020 23:01,approved 5 parishes hit replacements benefits ...,Neutral


In [ ]:
#Feature

In [8]:
cvectorizer = CountVectorizer(min_df=3, max_features=10000, tokenizer=lemmatize_text, ngram_range=(1,1),stop_words=STOPWORDS)

cvz = cvectorizer.fit_transform(df_Tpoics['text'])

n_topics = 6
n_iter = 100
lda_model = lda.LDA(n_topics=n_topics, n_iter=n_iter)
X_topics = lda_model.fit_transform(cvz)

INFO:lda:n_documents: 6836
INFO:lda:vocab_size: 4074
INFO:lda:n_words: 80598
INFO:lda:n_topics: 6
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -855488
INFO:lda:<10> log likelihood: -661107
INFO:lda:<20> log likelihood: -642355
INFO:lda:<30> log likelihood: -633619
INFO:lda:<40> log likelihood: -628150
INFO:lda:<50> log likelihood: -624530
INFO:lda:<60> log likelihood: -622074
INFO:lda:<70> log likelihood: -619570
INFO:lda:<80> log likelihood: -618569
INFO:lda:<90> log likelihood: -617496
INFO:lda:<99> log likelihood: -617008


In [9]:
# Topics in more detail below

n_top_words = 6
topic_summaries = []

topic_word = lda_model.topic_word_ 
vocab = cvectorizer.get_feature_names() 
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: help damage disaster need please affected
Topic 1: storm wind tropical landfall season rain
Topic 2: storm weather new video eye water
Topic 3: power day get not like back
Topic 4: tree safe power home stay please
Topic 5: power outage without crew damage customer


In [10]:
# Visualization to explore LDA topics using pyldavis
    
df_Tpoics['len_docs'] = df_Tpoics['text'].map(len)

In [11]:
ldadata = prepareLDAData()

In [12]:
pyLDAvis.enable_notebook()

In [13]:
prepared_data = pyLDAvis.prepare(**ldadata)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [19]:
pyLDAvis.save_html(prepared_data,'pyldadaviss.html')

In [1]:
#prepared_data

In [6]:
def lemmatize_text(tweets):  
    tweets = word_tokenize(tweets)
    valid_Words=[]
    for word in tweets :
        if word in STOPWORDS or word=='URL' or word=='user_mention' or word=='hurricanelaura' or word=='amp'or word=='delta'or word=='louisiana' or word=='laura' or word=='hurricane' or word=='zeta' or word=='hurricanedelta'or word=='amp' or word=='dsnap' or word=='nola'or 'hurricane' in word or len(word) <=2 :continue
        valid_Words.append(word)
        tweets=[lemma.lemmatize(word) for word in valid_Words]
        #tweets=' '.join(valid_Words)
    return tweets

In [7]:

def prepareLDAData():
    data = {
        'vocab': vocab,
        'doc_topic_dists': lda_model.doc_topic_,
        'doc_lengths': list(df_Tpoics['len_docs']),
        'term_frequency':cvectorizer.vocabulary_,
        'topic_term_dists': lda_model.components_
    } 
    return data

In [ ]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda.LDA(n_topics=n_topics, n_iter=n_iter), texts=lemmatize_text, dictionary=vocab, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Coherence Score:  0.3305274063700288